# Prepare Power Flow Data

This notebook presents the process of data preparation for building power flow regimes. The process is separated into stages (see, [DVC config](../dvc.yaml)):
 - "parse" --- extract necessary parameters from raw data
 - "prepare" --- combine and convert data to build final dataset
 - "model" --- build power flow cases.

The naming of variables in the final dataset corresponds to the [project convention](../convention.md).

Since the NREL-118 dataset is mostly intended for OPF tasks, it skips a lot of information that is not necessary to solve these kind of tasks. Thus, to append the data with missing info, [JEAS-118 dataset](http://motor.ece.iit.edu/data/JEAS_IEEE118.doc) is used as the primary source of NREL-118. The following sections describe all the stages of data processing and all the decisions made when preparing the final dataset.

In [1]:
import os

from src.data import parse_nrel118_buses
from src.data import prepare_buses

## Buses

To build a power flow model, the following information about buses is necessary:
- rated voltage level
- in service or out of service
- name (optional)
- region (optional)

In [2]:
# Load and parse bus data of NREL-118 power system
path_nrel118_buses = os.path.join(
    "..", "data", "raw", "nrel118", "additional-files-mti-118", "Buses.csv"
)
buses_nrel118 = parse_nrel118_buses(path_nrel118_buses)
buses_nrel118.head(2)

,name,region,load_participation_factor
0,bus_1,r1,0.047169
1,bus_2,r1,0.018496


The NREL-118 dataset contains only names and regions of buses ("load_participation_factor" is for load modelling). To add missing values, it is assumed the following:
- all buses are in service
- rated voltage level of buses 8, 9, 10, 26, 30, 38, 63, 64, 65, 68, 81, 116 equals to 345 kV, the rest of buses has the voltage level of 138 kV. This corresponds to transformers data from JEAS-118 dataset.

Thus, the final bus data looks as follows:

In [3]:
buses = prepare_buses(buses_nrel118)
buses.head(2)

,name,region,in_service,v_rated__kv
0,bus_1,r1,True,138
1,bus_2,r1,True,138
